# Calculate GWB

In [ ]:
import numpy as np

import zcode.inout as zio

import holodeck as holo
from holodeck.constants import YR, GYR
from holodeck import utils, cosmo, log
log.setLevel(log.WARNING)
import holodeck.sam_cython

import tqdm

TIME = 3 * GYR
DEF_NUM_FBINS = 7
DEF_PTA_DUR = 16.03     # [yrs]
SHAPE = 30

# Choose observed GW-Frequency bins based on nyquist sampling
fobs_edges = utils.nyquist_freqs_edges(DEF_PTA_DUR*YR, cad=0.1*YR)
fobs_edges = fobs_edges[:DEF_NUM_FBINS+1]
fobs_cents = utils.midpoints(fobs_edges)
fobs_orb_cents = fobs_cents / 2.0

with zio.timer.TimeIt("sam & static"):
    sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)
    sam.static_binary_density

with zio.timer.TimeIt("new method"):
    hard = holo.hardening.Fixed_Time_2PL_SAM(sam, TIME)
    gwb = sam.new_gwb(fobs_edges, hard)

with zio.timer.TimeIt("old method"):
    hard = holo.hardening.Fixed_Time_2PL.from_sam(sam, TIME)
    gwb = sam.test_gwb(fobs_edges, hard)



# Solving for Hardening Rate

In [ ]:
import numpy as np
import holodeck as holo
from holodeck.constants import YR, GYR, PC
from holodeck import utils, cosmo
import holodeck.sam_cython
import zcode.inout as zio

import tqdm

SHAPE = (12, 13, 14)
SHAPE = 30
TIME = 3 * GYR
SEPA_INIT = 1e3 * PC
RCHAR = 10.0 * PC
GAMMA_INNER = -1.0
GAMMA_OUTER = +1.5

sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)

with zio.timer.TimeIt("old hard"):
    old_hard = holo.hardening.Fixed_Time_2PL.from_sam(
        sam, TIME,
        sepa_init=SEPA_INIT, rchar=RCHAR, gamma_inner=GAMMA_INNER, gamma_outer=GAMMA_OUTER
    )
    old_norm = old_hard._norm.reshape(sam.shape)[:, :, 0]

with zio.timer.TimeIt("new hard"):
    new_hard = holo.hardening.Fixed_Time_2PL_SAM(
        sam, TIME,
        sepa_init=SEPA_INIT, rchar=RCHAR, gamma_inner=GAMMA_INNER, gamma_outer=GAMMA_OUTER
    )
    new_norm = new_hard._norm



In [ ]:
print(utils.stats(old_norm))
print(utils.stats(new_norm))
print(utils.stats(new_norm/old_norm))

# Test Binary Evolution

## Hard_GW

In [ ]:
print("Hello")

import numpy as np
from datetime import datetime
import holodeck as holo
from datetime import datetime
import holodeck.sam_cython
from holodeck.constants import YR, GYR, PC
from holodeck import utils, cosmo, log
log.setLevel(log.WARNING)

import zcode.inout as zio

DEF_NUM_FBINS = 7
DEF_PTA_DUR = 16.03     # [yrs]
SHAPE = (3, 4, 9)
# SHAPE = 30

# Choose observed GW-Frequency bins based on nyquist sampling
fobs_edges = utils.nyquist_freqs_edges(DEF_PTA_DUR*YR, cad=0.1*YR)
fobs_edges = fobs_edges[:DEF_NUM_FBINS+1]
fobs_cents = utils.midpoints(fobs_edges)
fobs_orb_cents = fobs_cents / 2.0

with zio.timer.TimeIt("static"):
    sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)
    sam.static_binary_density

hard = holo.hardening.Hard_GW

with zio.timer.TimeIt("new dynamic number"):
    new_redz_final, new_dnum = holodeck.sam_cython.dynamic_binary_number_at_fobs(fobs_orb_cents, sam, hard, cosmo)


### Compare against previous models

In [ ]:
# print("Hello")

import numpy as np
from datetime import datetime
import holodeck as holo
from datetime import datetime
import holodeck.sam_cython
from holodeck.constants import YR, GYR, PC, MSOL
from holodeck import utils, cosmo, log
log.setLevel(log.WARNING)

import zcode.inout as zio

DEF_NUM_FBINS = 7
DEF_PTA_DUR = 16.03     # [yrs]
SHAPE = 100
# SHAPE = (3, 4, 9)

# Choose observed GW-Frequency bins based on nyquist sampling
fobs_edges = utils.nyquist_freqs_edges(DEF_PTA_DUR*YR, cad=0.1*YR)
fobs_edges = fobs_edges[:DEF_NUM_FBINS+1]
fobs_cents = utils.midpoints(fobs_edges)
fobs_orb_cents = fobs_cents / 2.0

with zio.timer.TimeIt("static"):
    sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)
    sam.static_binary_density
    hard = holo.hardening.Hard_GW()

with zio.timer.TimeIt("new dynamic number"):
    new_redz_final, new_dnum = holodeck.sam_cython.dynamic_binary_number_at_fobs(fobs_orb_cents, sam, hard, cosmo)

with zio.timer.TimeIt("old dynamic number"):
    edges, old_dnum = sam.dynamic_binary_number(hard, fobs_orb_cents)
    old_redz_final = sam._redz_prime[:, :, :, np.newaxis] * np.ones_like(new_redz_final)
    
with zio.timer.TimeIt("test dynamic number"):
    edges, test_dnum, test_redz_final = sam.dynamic_binary_number_at_fobs(hard, fobs_orb_cents)
    


In [ ]:
val1 = new_redz_final
val2 = old_redz_final
val3 = test_redz_final
sel1 = (val1 > 0.0)
sel2 = (val2 > 0.0)
sel3 = (val3 > 0.0)
print(utils.frac_str(sel1))
print(utils.frac_str(sel2))
print(utils.frac_str(sel3))
print(utils.stats(val1[sel1]))
print(utils.stats(val2[sel2]))
print(utils.stats(val3[sel3]))
sel = sel1 & sel2 & sel3
print(utils.frac_str(sel))
print(utils.stats(val1[sel]/val2[sel]))
print(utils.stats(val3[sel]/val2[sel]))

In [ ]:
val1 = new_dnum
val2 = old_dnum
val3 = test_dnum
sel1 = (val1 > 0.0)
sel2 = (val2 > 0.0)
sel3 = (val3 > 0.0)
print(utils.frac_str(sel1))
print(utils.frac_str(sel2))
print(utils.frac_str(sel3))
print(utils.stats(val1[sel1]))
print(utils.stats(val2[sel2]))
print(utils.stats(val3[sel3]))
sel = sel1 & sel2 & sel3
print(utils.frac_str(sel))
print(utils.stats(val1[sel]/val2[sel]))
print(utils.stats(val1[sel]/val3[sel]))

## Fixed_Time_2PL_SAM

### Just make sure it works

In [ ]:
# print("Hello")

import numpy as np
from datetime import datetime
import holodeck as holo
from datetime import datetime
import holodeck.sam_cython
from holodeck.constants import YR, GYR, PC
from holodeck import utils, cosmo, log
log.setLevel(log.WARNING)

import zcode.inout as zio

DEF_NUM_FBINS = 7
DEF_PTA_DUR = 16.03     # [yrs]
SHAPE = (3, 4, 9)
SHAPE = 30
TIME = 3 * GYR

SEPA_INIT = 1e3 * PC
RCHAR = 10.0 * PC
GAMMA_INNER = -1.0
GAMMA_OUTER = +1.5

# Choose observed GW-Frequency bins based on nyquist sampling
fobs_edges = utils.nyquist_freqs_edges(DEF_PTA_DUR*YR, cad=0.1*YR)
fobs_edges = fobs_edges[:DEF_NUM_FBINS+1]
fobs_cents = utils.midpoints(fobs_edges)
fobs_orb_cents = fobs_cents / 2.0

with zio.timer.TimeIt("static"):
    sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)
    sam.static_binary_density

with zio.timer.TimeIt("new_hard"):
    new_hard = holo.hardening.Fixed_Time_2PL_SAM(
        sam, TIME,
        sepa_init=SEPA_INIT, rchar=RCHAR, gamma_inner=GAMMA_INNER, gamma_outer=GAMMA_OUTER,
        num_steps=20,
    )

with zio.timer.TimeIt("new dynamic number"):
    new_redz_final, new_dnum = holodeck.sam_cython.dynamic_binary_number_at_fobs(fobs_orb_cents, sam, new_hard, cosmo)


### Compare against previous calculations

In [ ]:
# print("Hello")

import numpy as np
from datetime import datetime
import holodeck as holo
from datetime import datetime
import holodeck.sam_cython
from holodeck.constants import YR, GYR, PC, MSOL
from holodeck import utils, cosmo, log
log.setLevel(log.WARNING)

import zcode.inout as zio

DEF_NUM_FBINS = 7
DEF_PTA_DUR = 16.03     # [yrs]
SHAPE = 30
SHAPE = (3, 4, 9)
TIME = 3 * GYR

SEPA_INIT = 1e3 * PC
RCHAR = 10.0 * PC
GAMMA_INNER = -1.0
GAMMA_OUTER = +1.5


# Choose observed GW-Frequency bins based on nyquist sampling
fobs_edges = utils.nyquist_freqs_edges(DEF_PTA_DUR*YR, cad=0.1*YR)
fobs_edges = fobs_edges[:DEF_NUM_FBINS+1]
fobs_cents = utils.midpoints(fobs_edges)
fobs_orb_cents = fobs_cents / 2.0

with zio.timer.TimeIt("static"):
    sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)
    sam.static_binary_density

with zio.timer.TimeIt("new_hard"):
    new_hard = holo.hardening.Fixed_Time_2PL_SAM(
        sam, TIME,
        sepa_init=SEPA_INIT, rchar=RCHAR, gamma_inner=GAMMA_INNER, gamma_outer=GAMMA_OUTER,
    )

with zio.timer.TimeIt("new dynamic number"):
    new_redz_final, new_dnum = holodeck.sam_cython.dynamic_binary_number_at_fobs(fobs_orb_cents, sam, new_hard, cosmo)

with zio.timer.TimeIt("old hard"):
    old_hard = holo.hardening.Fixed_Time_2PL.from_sam(
        sam, TIME,
        sepa_init=SEPA_INIT, rchar=RCHAR, gamma_inner=GAMMA_INNER, gamma_outer=GAMMA_OUTER
    )

with zio.timer.TimeIt("old dynamic number"):
    edges, old_dnum = sam.dynamic_binary_number(old_hard, fobs_orb_cents)
    old_redz_final = sam._redz_final


with zio.timer.TimeIt("test"):
    # edges, test_dnum, test_redz_final = sam.dynamic_binary_number_at_fobs(old_hard, fobs_orb_cents, 300)
    edges, test_dnum, test_redz_final = sam.dynamic_binary_number_at_fobs(
        old_hard, fobs_orb_cents, steps=300
    )


In [ ]:
NUM = int(1e5)
mt = np.random.uniform(4, 12, NUM)
mt = (10.0 ** mt) * MSOL
mr = 10.0**np.random.uniform(-4, 1, NUM)
sa = np.random.uniform(-6, -1, NUM)
sa = (10.0**sa) * PC

h1 = [holo.sam_cython.hard_gw(_mt, _mr, _sa) for _mt, _mr, _sa in zip(mt, mr, sa)]
h1 = np.asarray(h1)

m1, m2 = utils.m1m2_from_mtmr(mt, mr)
h2 = holo.utils.gw_hardening_rate_dadt(m1, m2, sa)
print(utils.stats(h1))
print(utils.stats(h2))
print(utils.stats(h1/h2))

nn = 1.0

new_hard_func = lambda x, y, z: holo.sam_cython.hard_func_2pwl_gw(
    x, y, z, nn, new_hard._rchar, new_hard._gamma_inner, new_hard._gamma_outer
)

h1 = [new_hard_func(_mt, _mr, _sa) for _mt, _mr, _sa in zip(mt, mr, sa)]
# h2 = old_hard._dadt_dedt(mt, mr, sa, nn, new_hard._rchar, new_hard._gamma_inner, new_hard._gamma_outer)[0]
h2 = old_hard._dadt_dedt(mt, mr, sa, nn, old_hard._rchar, old_hard._gamma_inner, old_hard._gamma_outer)[0]

print(utils.stats(h1))
print(utils.stats(h2))
print(utils.stats(h1/h2))

In [ ]:
sel1 = (new_redz_final > 0.0)
sel2 = (old_redz_final > 0.0)
sel3 = (test_redz_final > 0.0)
print(utils.stats(new_redz_final[sel1]))
print(utils.stats(old_redz_final[sel2]))
print(utils.stats(test_redz_final[sel3]))
print()
sel = sel1 & sel2 & sel3
print(utils.frac_str(sel1))
print(utils.frac_str(sel2))
print(utils.frac_str(sel3))
print()
print(utils.frac_str(sel))
print(utils.stats(new_redz_final[sel]/old_redz_final[sel]))
print(utils.stats(new_redz_final[sel]/test_redz_final[sel]))
print(utils.stats(test_redz_final[sel]/old_redz_final[sel]))

In [ ]:
sel1 = (new_dnum > 0.0)
sel2 = (old_dnum > 0.0)
sel3 = (test_dnum > 0.0)
print(utils.stats(new_dnum[sel1]))
print(utils.stats(old_dnum[sel2]))
print(utils.stats(test_dnum[sel3]))
print()
print(utils.frac_str(sel1))
print(utils.frac_str(sel2))
print(utils.frac_str(sel3))
sel = sel1 & sel2 & sel3
print(utils.frac_str(sel))
print()
weights = None
print(utils.stats(new_dnum[sel]/old_dnum[sel], weights=weights))
print(utils.stats(test_dnum[sel]/old_dnum[sel], weights=weights))
print(utils.stats(new_dnum[sel]/test_dnum[sel], weights=weights))
print()
weights = new_dnum[sel]
print(utils.stats(new_dnum[sel]/old_dnum[sel], weights=weights))
print(utils.stats(test_dnum[sel]/old_dnum[sel], weights=weights))
print(utils.stats(new_dnum[sel]/test_dnum[sel], weights=weights))

In [ ]:
# mt, mr = np.meshgrid(sam.mtot, sam.mrat, indexing='ij')
# sa = PC * (10.0 ** np.random.uniform(-4, -1, mt.shape))
# h1 = new_hard.dadt(mt, mr, sa)
# print(h1.shape)
# print(utils.stats(h1))

# mt, mr, _ = [mm.flatten() for mm in np.meshgrid(sam.mtot, sam.mrat, sam.redz, indexing='ij')]
# sa = np.stack(sam.redz.size*[sa], axis=-1).flatten()
# # sa = PC * (10.0 ** np.random.uniform(-4, -1, mt.shape))
# h1 = old_hard.dadt(mt, mr, sa)
# h1 = h1.reshape(sam.shape)[:, :, 0].flatten()
# print(h1.shape)
# print(utils.stats(h1))



# Integrate Differential-Number

In [ ]:
# print("Hello")

import numpy as np
from datetime import datetime
import holodeck as holo
from datetime import datetime
import holodeck.sam_cython
from holodeck.constants import YR, GYR, PC
from holodeck import utils, cosmo, log
log.setLevel(log.WARNING)

import zcode.inout as zio

DEF_NUM_FBINS = 7
DEF_PTA_DUR = 16.03     # [yrs]
# SHAPE = (3, 4, 9)
SHAPE = 30
TIME = 3 * GYR

SEPA_INIT = 1e3 * PC
RCHAR = 10.0 * PC
GAMMA_INNER = -1.0
GAMMA_OUTER = +1.5

# Choose observed GW-Frequency bins based on nyquist sampling
fobs_edges = utils.nyquist_freqs_edges(DEF_PTA_DUR*YR, cad=0.1*YR)
fobs_edges = fobs_edges[:DEF_NUM_FBINS+1]
fobs_cents = utils.midpoints(fobs_edges)
fobs_orb_cents = fobs_cents / 2.0

sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)
sam.static_binary_density

old_hard = holo.hardening.Fixed_Time_2PL.from_sam(
    sam, TIME,
    sepa_init=SEPA_INIT, rchar=RCHAR, gamma_inner=GAMMA_INNER, gamma_outer=GAMMA_OUTER
)

old_edges, old_dnum = sam.dynamic_binary_number(old_hard, fobs_orb_cents, zero_stalled=True, zero_coalesced=True)
old_redz_final = sam._redz_final

In [ ]:
old_edges[-1] = fobs_edges
old_number = utils._integrate_grid_differential_number(old_edges, old_dnum, freq=False)
old_number = old_number * np.diff(np.log(fobs_edges))
print(old_number.shape)

In [ ]:
def integrate_bins(edges, dnum):
    edge_shape = list(dnum.shape)
    cent_shape = [ss-1 for ss in edge_shape]
    # frequency dimension stays the same
    cent_shape[-1] = edge_shape[-1]

    mtot_log10 = np.log10(edges[0])
    dlog_freq = np.diff(np.log(edges[-1]))

    n_mtot, n_mrat, n_redz, n_freq = cent_shape
    numb = np.zeros(cent_shape)
    
    for mm in range(n_mtot):
        dm = mtot_log10[mm+1] - mtot_log10[mm]
        for qq in range(n_mrat):
            dq = edges[1][qq+1] - edges[1][qq]
            dmdq = dm * dq
            for zz in range(n_redz):
                dz = edges[2][zz+1] - edges[2][zz]
                dmdqdz = dmdq * dz
                for ff in range(n_freq):
                    temp = 0.0
                    for ii in range(2):
                        for jj in range(2):
                            for kk in range(2):
                                temp += dnum[mm+ii, qq+jj, zz+kk, ff]
                    numb[mm, qq, zz, ff] = temp * dmdqdz * dlog_freq[ff] / 8.0
            
    return numb

numb = integrate_bins(old_edges, old_dnum)

In [ ]:
new_numb = holo.sam_cython.integrate_differential_number_3dx1d(old_edges, old_dnum)

In [ ]:
sel1 = (old_number > 0.0)
sel2 = (numb > 0.0)
sel3 = (new_numb > 0.0)
print(utils.frac_str(sel1))
print(utils.frac_str(sel2))
print(utils.frac_str(sel3))
print()
print(utils.stats(old_number[sel1]))
print(utils.stats(numb[sel2]))
print(utils.stats(new_numb[sel3]))
print()
sel = sel1 & sel2 & sel3
print(utils.stats(numb[sel]/old_number[sel]))
print(utils.stats(new_numb[sel]/old_number[sel]))